Install these library on TERMINAL if you haven't install

pip install matplotlib tensorflow opency-python scikit-learn scikit-image pillow pandas


For Window Linux Subsystem user:
1. Display plot from matplotlib require Xming Server running.
2. Download (Xming)[https://sourceforge.net/projects/xming/]
3. Update .bashrc
    vim ~/.bashrc
4. Append at the end of the file
    export DISPLAY=:0



In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import json
import os
import cv2

In [ ]:
from PIL import Image
from skimage.feature import hog
from skimage.color import rgb2grey

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc

In [ ]:
# check for tkinter
if os.environ.get('DISPLAY','') == '':
    print('no display found. Using non-interactive Agg backend')
    mpl.use('Agg')

In [ ]:
# Global variables
# parent_dir and model_dir should be change according 
# the dataset location and location of save models according to users 
parent_dir = "/mnt/c/Users/nhmin/Downloads/food-101/"
bin_n = 16 # Number of bin
project_dir = os.getcwd()
model_dir = "/mnt/c/Users/nhmin/Downloads/"
# List of class take from the dataset
class_label = ["pad_thai", "pho", "ramen", "spaghetti_bolognese", "spaghetti_carbonara"]

`get_image()` taken in image path and return numpy 384x384 array of the image.
`load_json(path)` return a dictionary of classes as key and list of image directories as value. This is load from **train.json** and **test.json** in the **meta** folder from the dataset.

In [ ]:
# List of class take from the dataset
class_label = ["pad_thai", "pho", "ramen", "spaghetti_bolognese", "spaghetti_carbonara"]

def get_image(path):
    # image size = 384x384
    img = Image.open(path + ".jpg")
    resized_image = img.resize((384,384), Image.ANTIALIAS)
    return np.array(resized_image)


def load_json(path):
    final_data = dict()
    # Load in json file to create dictionary: key = class label; value = file path
    with open(path, 'r') as file:
        data = json.load(file)
    for label in class_label:
        final_data.update({label : data.get(label)})
    return final_data

I started off with loading data from **train.json** into `dictionary{label:[image_dir]`. We will use (Histogram of Oriented Gradient (HOG))[https://scikit-image.org/docs/dev/api/skimage.feature.html#skimage.feature.hog] of every image flatten into 1-D array and form a new return data as `dictionary{label: [image_name,[hog_array]]}`.

In [ ]:
def label_processing(data, label_dictionary):
    final_label = []
    for i in label_dictionary.keys():
        label_value = label_dictionary.get(i)
        for image in data.get(i):
            final_label.append(label_value)
    return np.array(final_label)

In [ ]:
def load_HOG_data(data):
    feature_lists = []
    for image in data:
            file_name = image.split('/')[1]
            file_image = get_image(parent_dir + "images/" + image)
            # given 32x32 cell
            image_feature, image_hog = hog(file_image, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(8, 8), block_norm = 'L2-Hys', visualize=True, multichannel=True)
            feature_lists.append(image_feature)
    return np.array(feature_lists)

def feature_format(data):
    feature_matrix = load_HOG_data(data)
    ss = StandardScaler()
    food_stand = ss.fit_transform(feature_matrix)
    pca = PCA(n_components = feature_matrix.shape[0])
    food_pca = ss.fit_transform(food_stand)
    return food_pca

def pre_process_data(data_json):
    pca_data = dict()
    labels = list(data_json.keys())
    for label in labels:
        pca_data.update({label : feature_format(data_json.get(label))})
    return pca_data

In [ ]:
def train_dataset(train_json):
    # Getting all the label and combination of all label
    labels = list(train_json.keys())
    class_combinations = list(combinations(labels, 2))

    #Get all pca dictinary data for both train and test
    train_pca_dict = pre_process_data(train_json)
    for combination in class_combinations:
        #SVM classification
        svm = SVC(gamma='auto', kernel='linear', probability=True)
        combine_train_data = np.vstack((train_pca_dict.get(combination[0]), train_pca_dict.get(combination[1])))
        label_dict = {combination[0] : 1, combination[1] : -1}
        label_lists_train = label_processing(train_json, label_dict)

        x_train = pd.DataFrame(combine_train_data)
        y_train = pd.Series(label_lists_train)

        svm.fit(x_train, y_train)
        filename = combination[0] + "_" + combination[1] + "_model.sav"
        model_filename = model_dir + "/models_save/" + filename
        pickle.dump(svm, open(model_filename, 'wb'))